In [1]:
# pip install pandas
# pip install numpy
# !pip install transformers -U
# pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
# pip install accelerate -U
# pip install transformers[torch]
# pip install accelerate>=0.21.0

In [1]:
import pandas as pd
data = pd.read_csv(r"D:\FYP\Final Model Training\Data Set\Final_data_set_shuffle_for_model.csv")
data.head()

,Headline,Label
0,bBelgian cardinals prevent priests to be in a ...,0
1,Majority of Russians and Americans View Each O...,0
2,Mr. Koistinen joins from Nokia Siemens Network...,0
3,Blatter to stand again for footballs top job,0
4,Brazils Army Moves To Protect Indigenous Aw Tr...,1


In [2]:
data["Label"].value_counts()
# data.columns

Label
1    62874
0    61492
Name: count, dtype: int64

In [3]:
data = data[['Headline','Label']]
data.Label.value_counts()

Label
1    62874
0    61492
Name: count, dtype: int64

In [4]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score
import torch
from transformers import TrainingArguments,Trainer
from transformers import BertTokenizer,BertForSequenceClassification

In [9]:
tokenizer = BertTokenizer.from_pretrained('google-bert/bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('google-bert/bert-base-uncased',num_labels=2)
model = model.to('cpu') # if CPU is available
# model = model.to('cuda') # if GPU is available

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
# padding  = Ture Make all the list at same list add extra 0 to the list
# truncation = Ture which wil be truncate any text which has more than 512 words. Which maximum context size is the maximun amount of words a model can accept  
# return_tensors=True Convert normal python list to the tensorflow tensor
sample_data = ["Hello World","My name is Arsalan Bakhtiar"]
tokenizer(sample_data ,padding = True , truncation= True , max_length= 512)

{'input_ids': [[101, 7592, 2088, 102, 0, 0, 0, 0, 0, 0, 0, 0], [101, 2026, 2171, 2003, 29393, 7911, 2078, 8670, 10023, 10711, 2099, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [11]:
X = list(data['Headline'])
y = list(data['Label'])
X_train,X_val,y_train,y_val = train_test_split(X,y,test_size = 0.2,stratify=y)
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length= 512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)

In [12]:
len(X_train),len(X_val)

(99492, 24874)

In [13]:
X_train_tokenized.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [14]:
print(X_train_tokenized['input_ids'][0])

[101, 4518, 3863, 2713, 9800, 11387, 14526, 14840, 2624, 9626, 2038, 2405, 2049, 3296, 3189, 1998, 3361, 8635, 2005, 2230, 1998, 2049, 2034, 5971, 5368, 3189, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [15]:
sample_data = ["Hello World","My name is Arsalan Bakhtiar"]
tokenizer(sample_data ,padding = True , truncation= True , max_length= 512)

{'input_ids': [[101, 7592, 2088, 102, 0, 0, 0, 0, 0, 0, 0, 0], [101, 2026, 2171, 2003, 29393, 7911, 2078, 8670, 10023, 10711, 2099, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [16]:
# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [17]:
train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)

In [18]:
# train_dataset[5]

In [19]:
def compute_metrics(p):
    print(type(p))
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [20]:
# Define Trainer
args = TrainingArguments(
    output_dir="output",
    num_train_epochs = 4,
    per_device_train_batch_size=8

)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

Step,Training Loss


In [ ]:
trainer.evaluate()

In [ ]:
np.set_printoptions(suppress=True)

In [ ]:
text = "In the third quarter of 2010 , net sales increased by 5.2 % to EUR 205.5 mn , and operating profit by 34.9 % to EUR 23.5 mn"
inputs = tokenizer(text,padding = True, truncation = True, return_tensors='pt').to('cpu')
outputs = model(**inputs)
# print(outputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
# print(predictions)
predictions = predictions.cpu().detach().numpy()
predictions

In [ ]:
trainer.save_model("google-bert/bert-base-uncased-512-4 epoch")

In [ ]:
import torch
from transformers import BertTokenizer,BertForSequenceClassification
model_2 = BertForSequenceClassification.from_pretrained("Fine_tuned_model")
model_2.to('cpu')

In [4]:
import torch
# text = "That was good point"
text = "For the last quarter of 2010 , Componenta 's net sales doubled to EUR131m from EUR76m for the same period a year earlier , while it moved to a zero pre-tax profit from a pre-tax loss of EUR7m "
tokenizer = BertTokenizer.from_pretrained('google-bert/bert-base-uncased')
inputs = tokenizer(text,padding = True, truncation = True, return_tensors='pt').to('cpu')
outputs = model_2(**inputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
predictions = predictions.cpu().detach().numpy()
predictions
x = predictions[0, 0]
y = predictions[0, 1]
print(predictions)
print("X :",x)
print("Y :",y)

[[0.01005797 0.98994195]]
X : 0.010057974
Y : 0.98994195


In [ ]:
# first one is 0 and second 1 is 1

In [11]:
print(predictions)
print("X :",x)
print("Y :",y)

[[0.01005797 0.98994195]]
X : 0.010057974
Y : 0.98994195


In [ ]:
from transformers import TFAutoModel,
                         AutoTokenizer 

In [ ]:
# Download the model form the internet
model = TFAutoModel.from_pretrained("bert-base-uncased")

In [ ]:
# Tokenizer
tokenizer = TFAutoModel.from_pretrained("bert-base-uncased")







In [ ]:
# padding  = Ture Make all the list at same list add extra 0 to the list
# truncation = Ture which wil be truncate any text which has more than 512 words. Which maximum context size is the maximun amount of words a model can accept  
# return_tensors=True Convert normal python list to the tensorflow tensor
inputs = tokenizer("Hello World",'Hi how are',padding = True, truncation = True, return_tensors = True)

In [ ]:
# call bert to train the model
output = model(inputs)